# Bilan hydrique à partir d'observations Météo-France horaires qualifiées

## Reconstruction des données météorologiques pour une station de référence

### Définition des paramètres

In [1]:
# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]
REF_STATION_ALTITUDE = 50.

# Départements où chercher des stations
ID_DEPARTEMENTS = [35, 50]

NN_NOMBRE = None
# NN_RAYON_KM = 20.
NN_RAYON_KM = 35.

# Identification de l'API Météo-France
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

### Lecture de la liste des stations

In [2]:
import meteofrance

# Météo-France API
METEOFRANCE_API = 'DPClim'

# Fréquence des données climatiques
METEOFRANCE_FREQUENCE = 'horaire'

# Initialisation d'un client pour accéder à l'API Météo-France
client = meteofrance.Client(METEOFRANCE_API, application_id=APPLICATION_ID)

In [3]:
import pandas as pd

LECTURE_LISTE_STATIONS = True

# Météo-France API
METEOFRANCE_API = 'DPClim'

# Fréquence des données climatiques
METEOFRANCE_FREQUENCE = 'horaire'

l_listes = []
for id_dep in ID_DEPARTEMENTS:
    filepath_liste_stations = meteofrance.get_filepath_liste_stations(
        client, frequence=METEOFRANCE_FREQUENCE, id_departement=id_dep)
    
    if LECTURE_LISTE_STATIONS:
        # Lecture de la liste des stations par département
        df_liste_stations_dep = pd.read_csv(
            filepath_liste_stations, index_col=client.id_station_label)
    else:
        # Demande de la liste des stations pour le département
        section = meteofrance.SECTION_LISTE_STATIONS
        params = {'id-departement': id_dep}
        response = meteofrance.demande(
            client, section, params=params, frequence=METEOFRANCE_FREQUENCE)
        df_liste_stations_dep = meteofrance.response_text_to_frame(
            client, response, index_col=client.id_station_label)

        # Sauvegarde de la liste des stations par département
        df_liste_stations_dep.to_csv(filepath_liste_stations)

    # Liste pour compilation
    l_listes.append(df_liste_stations_dep)

# Compilation
df_liste_stations_brute = pd.concat(l_listes, axis='index')

# Garder les stations valides seulement
df_liste_stations = meteofrance.filtrer_stations_valides(client, df_liste_stations_brute)

### Sélection des plus proches voisins de la station de référence

In [4]:
import geo

df_liste_stations_nn = geo.selection_plus_proches_voisins(
    df_liste_stations, REF_STATION_LATLON, client.latlon_labels,
    nombre=NN_NOMBRE, rayon_km=NN_RAYON_KM)

df_liste_stations_nn

,nom,typePoste,lon,lat,alt,distance
id,,,,,,
35044001,BROUALAN,2,-1.640833,48.485667,99,6
50410003,PONTORSON,1,-1.505167,48.585667,33,9
35224001,PLERGUER,2,-1.843667,48.524833,30,17
35110003,FEINS SA,1,-1.596833,48.326833,87,24
35225001,PLESDER,2,-1.924833,48.406833,56,27
35178001,MEZIERES-SUR-C.,2,-1.439000,48.308833,71,29
50531001,ST OVIN,2,-1.248667,48.682500,155,31
50218001,GRANVILLE,3,-1.613667,48.834500,37,33
35228001,DINARD,0,-2.076333,48.584833,65,34


### Obtention des données météorologiques pour les stations voisines

In [5]:
import bilan
import etp

DATE_DEB_PERIODE = '2022-01-01T00:00:00Z'
DATE_FIN_PERIODE = '2024-12-31T23:00:00Z'

LECTURE_DONNEE = False

# Dates délimitant des périodes d'une année subdivisant la période totale
idx_dates_deb = pd.date_range(
    start=DATE_DEB_PERIODE, end=DATE_FIN_PERIODE, freq='YS-JAN')
idx_dates_fin = pd.date_range(
    start=DATE_DEB_PERIODE, end=DATE_FIN_PERIODE, freq='YE-DEC') + pd.Timedelta(hours=23)
list_dates_deb = [d.isoformat().replace("+00:00", "Z") for d in idx_dates_deb]
list_dates_fin = [d.isoformat().replace("+00:00", "Z") for d in idx_dates_fin]

# Variables utilisées pour le calcul de l'ETP et du bilan hydrique 
variables_pour_calculs = dict(**etp.VARIABLES_CALCUL_ETP,
                              **bilan.VARIABLES_CALCUL_BILAN)
variables_pour_calculs_sans_etp = variables_pour_calculs.copy()
del variables_pour_calculs_sans_etp['etp']
    
df_meteo = pd.DataFrame(dtype=float)
for date_deb, date_fin in zip(list_dates_deb, list_dates_fin):
    filepath_donnee_an = meteofrance.get_filepath_donnee_periode(
        client, date_deb, date_fin, frequence=METEOFRANCE_FREQUENCE,
        df_liste_stations=df_liste_stations_nn)
    
    if LECTURE_DONNEE:
        # Lecture des données des stations pour la période
        df_meteo_an = pd.read_csv(
            filepath_donnee_an, parse_dates=[client.time_label],
            index_col=[client.id_station_donnee_label, client.time_label])
    else:
        # Demande des données des stations pour la période
        variables = [client.variables_labels[METEOFRANCE_FREQUENCE][k]
                     for k in variables_pour_calculs_sans_etp]
        df_meteo_an = meteofrance.compiler_telechargement_des_stations_periode(
            client, df_liste_stations_nn, date_deb, date_fin,
            frequence=METEOFRANCE_FREQUENCE,
            read_csv_kwargs={'date_format': "%Y%m%d%H"})[variables]
    
        # Sauvegarde des données des stations pour la période par département
        df_meteo_an.to_csv(filepath_donnee_an)

    # Compilation des années
    df_meteo = pd.concat([df_meteo, df_meteo_an], axis='index')

Received status code 204. Retrying...
Received status code 204. Retrying...
Received status code 204. Retrying...
Received status code 204. Retrying...
Received status code 204. Retrying...
Received status code 204. Retrying...
Received status code 204. Retrying...


### Interpolation des données météorologiques à la station de référence

Les variables sont également renommées en utilisant des noms communs à l'ensemble de ces notebooks quelque soit l'API utilisée.

In [6]:
LECTURE_DONNEE_REF_HEURE = False

filepath_donnee = meteofrance.get_filepath_donnee_periode(
    client, DATE_DEB_PERIODE, DATE_FIN_PERIODE,
    frequence=METEOFRANCE_FREQUENCE,
    df_liste_stations=df_liste_stations_nn)
str_ref_station_name = REF_STATION_NAME.lower().replace(' ', '')
filepath_donnee_ref_heure = filepath_donnee.with_name(
    filepath_donnee.stem + '_' + str_ref_station_name + filepath_donnee.suffix)

if LECTURE_DONNEE_REF_HEURE:
    # Lecture des données horaires des stations pour la période
    df_meteo_ref_heure = pd.read_csv(
        filepath_donnee_ref_heure, parse_dates=[client.time_label],
        index_col=client.time_label)
else:
    # Demande des données horaires des stations pour la période
    df_meteo_ref_heure = geo.interpolation_inverse_distance_carre(
        df_meteo, df_liste_stations_nn['distance'])
    
    # Sauvegarde des données horaires des stations pour la période
    df_meteo_ref_heure.to_csv(filepath_donnee_ref_heure)

df_meteo_ref_heure = meteofrance.renommer_variables(
    client, df_meteo_ref_heure, METEOFRANCE_FREQUENCE)

### Conversion des unités

In [7]:
df_meteo_ref_heure_si = meteofrance.convertir_unites(
    client, df_meteo_ref_heure)

### Estimation de l'ETP journalière pour la station de référence

In [8]:
df_meteo_ref_heure_si['etp'] = etp.calcul_etp(
    df_meteo_ref_heure_si, *REF_STATION_LATLON, REF_STATION_ALTITUDE)

#### Aggrégation à l'échelle journalière

In [9]:
LECTURE_DONNEE_REF = False

filepath_donnee_ref = filepath_donnee_ref_heure.with_name(
    filepath_donnee_ref_heure.name.replace(
        METEOFRANCE_FREQUENCE, 'quotidienne_estimee').replace(
        '230000', '000000'))

if LECTURE_DONNEE_REF:
        # Lecture des données journalières des stations pour la période
     df_meteo_ref_si = pd.read_csv(
        filepath_donnee_ref, parse_dates=[client.time_label],
        index_col=client.time_label)
else:
    # Calcul des valeurs journalières des variables météo
    df_meteo_ref_si = pd.DataFrame(dtype=float)
    for variable, series in df_meteo_ref_heure_si.items():
        df_meteo_ref_si.loc[:, variable] = getattr(
            df_meteo_ref_heure_si[variable].resample('D'),
            variables_pour_calculs[variable])()
    
    # Sauvegarde des données journalières des stations pour la période
    df_meteo_ref_si.to_csv(filepath_donnee_ref)

df_meteo_ref_si

,vitesse_vent_10m,temperature_2m,humidite_relative,rayonnement_global,precipitation,etp
DATE,,,,,,
2022-01-01 00:00:00+00:00,5.481749,284.933671,0.848411,3650000.0,0.208127,0.999094
2022-01-02 00:00:00+00:00,6.032281,285.352880,0.834083,3270000.0,0.325982,1.109826
2022-01-03 00:00:00+00:00,4.594824,283.599015,0.929913,1220000.0,20.106851,0.445573
2022-01-04 00:00:00+00:00,4.785213,280.170804,0.897131,2360000.0,4.140486,0.556847
2022-01-05 00:00:00+00:00,3.654604,277.288894,0.752369,1970000.0,1.846153,0.790548
...,...,...,...,...,...,...
2024-12-27 00:00:00+00:00,1.675761,277.653392,0.939435,1630000.0,0.000000,0.256063
2024-12-28 00:00:00+00:00,1.926316,275.089490,0.963579,900000.0,0.241090,0.168771
2024-12-29 00:00:00+00:00,2.865470,276.825644,0.958136,870000.0,0.000000,0.229901


## Estimation du bilan hydrique

### Définition des paramètres

In [10]:
# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.1

# Choix de la texture
TEXTURE = 'Terres limoneuses'

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_RU_REMPLIE = 1.

# Besoin d'irrigation minimal à partir du quel irriguer (mm)
SEUIL_IRRIGATION = 1.

# Conversion de hauteur (mm) vers durée d'irrigation (min)
HAUTEUR_VERS_DUREE_IRRIGATION = 10

### Calcul du bilan hydrique

In [11]:
# Choix de la culture
CULTURE = list(bilan.KC)[0]

# Choix du stade
STADE = list(bilan.KC[CULTURE])[0]

df_bilan = bilan.calcul_bilan(
    df_meteo_ref_si,
    TEXTURE, FRACTION_CAILLOUX,
    CULTURE, STADE,
    FRACTION_RU_REMPLIE, RU_VERS_RFU,
    seuil_irrigation=SEUIL_IRRIGATION,
    hauteur_vers_duree_irrigation=HAUTEUR_VERS_DUREE_IRRIGATION)

df_bilan.describe()

,etp,profondeur_enracinement,profondeur_terrefine,ru,rfu,precipitation,etm_culture,besoin_irrigation,rfu_cible,duree_irrigation
count,1096.000000,1096.0,1096.0,1.096000e+03,1.096000e+03,1096.000000,1096.000000,1096.000000,1.096000e+03,1096.000000
mean,-2.216805,20.0,18.0,3.240000e+01,2.160000e+01,2.270448,-1.108402,-1.162045,2.160000e+01,5.875688
std,1.487764,0.0,0.0,7.108671e-15,3.554336e-15,4.231072,0.743882,4.476637,3.554336e-15,9.267763
min,-8.851211,20.0,18.0,3.240000e+01,2.160000e+01,0.000000,-4.425605,-40.135985,2.160000e+01,0.000000
25%,-3.181343,20.0,18.0,3.240000e+01,2.160000e+01,0.055105,-1.590671,-1.900882,2.160000e+01,0.000000
50%,-1.900004,20.0,18.0,3.240000e+01,2.160000e+01,0.293936,-0.950002,0.280774,2.160000e+01,0.000000
75%,-0.974633,20.0,18.0,3.240000e+01,2.160000e+01,2.651132,-0.487316,1.327109,2.160000e+01,13.271094
max,-0.168771,20.0,18.0,3.240000e+01,2.160000e+01,40.275084,-0.084385,4.425605,2.160000e+01,44.256055
